In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minzzzzi","key":"74a980fb52033cf0ead89c84120a4a9c"}'}

In [2]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle

Uninstalling kaggle-1.5.8:
  Successfully uninstalled kaggle-1.5.8
     |████████████████████████████████| 1.5MB 8.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 59 kB 3.4 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73274 sha256=3687077084778dd1207657c181a4b531839698ae371905d1659178622b849527
  Stored in directory: /root/.cache/pip/wheels/cf/aa/f0/ed1179bbcd729b29d0dfda59826fb3b55f0a4a0c3f713c1c82
Successfully built kaggle


In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c 2020-abalone-age

  0% 0.00/18.0k [00:00<?, ?B/s]
100% 18.0k/18.0k [00:00<00:00, 14.2MB/s]


In [5]:
!unzip 2020-abalone-age.zip

Archive:  2020-abalone-age.zip
  inflating: 2020-abalone-submit.csv  
  inflating: 2020-abalone-test.csv   
  inflating: 2020-abalone-train.csv  


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [16]:
train = pd.read_csv('2020-abalone-train.csv',header=None,skiprows=1)
test = pd.read_csv('2020-abalone-test.csv',header=None,skiprows=1)
submit = pd.read_csv('2020-abalone-submit.csv')

train[:5]

,0,1,2,3,4,5,6,7,8
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [17]:
num = {'M':0,'F':1,'I':2}
train[0] = train[0].map(num)
test[0] = test[0].map(num)
train[:5]

,0,1,2,3,4,5,6,7,8
0,0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,2,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [47]:
x_train = train.loc[:,0:7]
y_train = train.loc[:,8]

x_test = test.loc[:,0:7]

In [48]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

x_train_std = sc.fit_transform(x_train)
x_test_std = sc.fit_transform(x_test)

---
#KNN
---

In [49]:
from sklearn.neighbors import KNeighborsRegressor
KNN = KNeighborsRegressor(n_neighbors = 3, weights = 'distance', p = 2)
'''
KNN.fit(x_train,y_train)
train_pred = KNN.predict(x_train)
print(mean_squared_error(y_train,train_pred))
'''
KNN.fit(x_train_std,y_train)
train_pred = KNN.predict(x_train_std)
print(mean_squared_error(y_train,train_pred))

0.0


In [45]:
test_pred = KNN.predict(x_test_std)

for i in range(len(test_pred)):
  submit['Predict'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True, index = None)

In [46]:
!kaggle competitions submit -c 2020-abalone-age -f submission.csv -m "KNN"

100% 1.17k/1.17k [00:03<00:00, 316B/s]
Successfully submitted to 2020-abalone-age

#### (n_neighbors = 5, weights = 'distance', p = 2) + no std / 3.64
#### (n_neighbors = 5, weights = 'distance', p = 2) + std / 3.60
#### (n_neighbors = 3, weights = 'distance', p = 2) + std / 3.70



---
#Dicision Tree
---

In [59]:
from sklearn.tree import DecisionTreeRegressor
dtc = DecisionTreeRegressor(random_state = 1)

dtc.fit(x_train,y_train)
train_pred = dtc.predict(x_train)
print(mean_squared_error(y_train,train_pred))

0.0


In [60]:
test_pred = dtc.predict(x_test)

for i in range(len(test_pred)):
  submit['Predict'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True, index = None)
!kaggle competitions submit -c 2020-abalone-age -f submission.csv -m "dtc"

100% 1.17k/1.17k [00:05<00:00, 227B/s]
Successfully submitted to 2020-abalone-age

####std / 4.31277
####max_depth = 5 + std/ 4.30
####original + max_depth = 5/ 3.98
####original / 4.31798

---
#Logistic Regression
---

In [75]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C = 1.0,random_state = 1, solver ='saga',max_iter=10000)

logit.fit(x_train_std,y_train)
train_pred = logit.predict(x_train_std)
print(mean_squared_error(y_train,train_pred))

8.4951171875


In [76]:
test_pred = logit.predict(x_test_std)

for i in range(len(test_pred)):
  submit['Predict'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True, index = None)
!kaggle competitions submit -c 2020-abalone-age -f submission.csv -m "logit"

100% 1.17k/1.17k [00:04<00:00, 296B/s]
Successfully submitted to 2020-abalone-age

5.64
####(C = 10.0,random_state = 1, solver ='saga',max_iter=10000) + std / 4.04
####(C = 1.0,random_state = 1, solver ='saga',max_iter=10000) + std / 3.90